# BARANOV ANDREY. MDS Spring 2023 Cohort SGA . User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


# Spark DF solution

In [1]:
# setting up environment
import findspark
findspark.init()

In [2]:
# spark imports
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType

In [3]:
# Initialize SparkContext and SparkSession
sc = pyspark.SparkContext(appName='jupyter')
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-27 18:39:17,187 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
# loading the CSV file into a DataFrame
clickstream_df = spark.read.csv('/data/clickstream.csv', header=True, sep = '\t', inferSchema=True)

In [5]:
# showing the data structure to understand columns and data types
clickstream_df.printSchema()
clickstream_df.show(5)


root
 |-- user_id: integer (nullable = true)
 |-- session_id: integer (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_page: string (nullable = true)
 |-- timestamp: integer (nullable = true)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
+-------+----------+------------+----------+----------+
only showing top 5 rows



In [6]:
# Create a window specification for each session, ordered by the timestamp
w_spec = Window.partitionBy("user_id", "session_id").orderBy("timestamp")

In [7]:
# Determine the timestamp of the first error occurrence within each session
clickstream_df_with_errors = clickstream_df.withColumn("error_timestamp", F.min(F.when(F.col("event_type").like("%error%"), F.col("timestamp"))).over(w_spec))

In [8]:
# Exclude rows that occur at or after the identified error timestamp
clickstream_filtered = clickstream_df_with_errors.filter((F.col("error_timestamp").isNull()) | (F.col("timestamp") < F.col("error_timestamp")))

In [9]:
# Order events within each session by user ID, session ID, and timestamp
clickstream_sorted = clickstream_filtered.orderBy("user_id", "session_id", "timestamp")

In [10]:
# User Defined Function (UDF) to eliminate consecutive duplicates
def eliminate_consecutive_duplicates(event_pages):
    if not event_pages:
        return ""
    deduplicated_pages = [event_pages[0]]
    for page in event_pages[1:]:
        if page != deduplicated_pages[-1]:
            deduplicated_pages.append(page)
    return '-'.join(deduplicated_pages)

In [11]:
eliminate_consecutive_udf = F.udf(eliminate_consecutive_duplicates, StringType())

In [12]:
# Aggregate by session, collect event pages, remove consecutive duplicates, and generate routes
clickstream_routes = clickstream_sorted.groupBy("user_id", "session_id").agg(eliminate_consecutive_udf(F.collect_list("event_page")).alias("route"))

In [13]:
# Count each unique route's occurrences and sort the results in descending order
clickstream_routes_counts = clickstream_routes.groupBy("route").count().orderBy(F.desc("count"))

In [14]:
# Retrieve the top 30 most frequent routes
result = clickstream_routes_counts.limit(30)

In [15]:
# Show the final results
result.show()

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  138|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
+--------------------+-----+
only showing top 20 rows



In [17]:
import json

In [20]:
# Collect results to a list of tuples
result_list = result.collect()

# Format the results as a dictionary for JSON
results_dict = {route: count for route, count in result_list}

# Save the results to a JSON file
with open('result.json', 'w') as json_file:
    json.dump(results_dict, json_file)


In [21]:
#!curl -F file=@result.json 51.250.123.136:80/MDS-LSML1/nndrew/w6/2

0.9999999999999999
Correct main answer!
Correct main-archive answer!
Correct main-rabota answer!
Correct main-internet answer!
Correct main-bonus answer!
Correct main-news answer!
Correct main-tariffs answer!
Correct main-online answer!
Correct main-vklad answer!
Correct main-rabota-archive answer!
Wrong main-archive-rabota answer!
Wrong main-bonus-archive answer!
Wrong main-rabota-bonus answer!
Wrong main-news-rabota answer!
Wrong main-bonus-rabota answer!
Wrong main-archive-internet answer!
Wrong main-rabota-news answer!
Wrong main-internet-rabota answer!
Wrong main-archive-news answer!
Wrong main-rabota-internet answer!
Wrong main-internet-archive answer!
Wrong main-archive-bonus answer!
Wrong main-internet-bonus answer!
Wrong main-tariffs-internet answer!
Wrong main-news-archive answer!
Wrong main-news-internet answer!
Wrong main-archive-tariffs answer!
Wrong main-internet-news answer!
Wrong main-tariffs-archive answer!
Wrong main-rabota-main answer!


In [22]:
spark.stop()

# Spark SQL solution

In [23]:
# Setting up environment
import findspark
findspark.init()

# Spark imports
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

# Initialize SparkContext and SparkSession
sc = pyspark.SparkContext(appName='jupyter')
spark = SparkSession(sc)

# Load the CSV file into a DataFrame
clickstream_df = spark.read.csv('/data/clickstream.csv', header=True, sep='\t', inferSchema=True)

# Create a temporary view for SQL queries
clickstream_df.createOrReplaceTempView("clickstream")

# Determine the timestamp of the first error occurrence within each session
error_timestamp_query = """
SELECT user_id,
       session_id,
       timestamp,
       event_type,
       event_page,
       MIN(CASE WHEN event_type LIKE '%error%' THEN timestamp END) OVER (PARTITION BY user_id, session_id) AS error_timestamp
FROM clickstream
"""

error_timestamp_df = spark.sql(error_timestamp_query)
error_timestamp_df.createOrReplaceTempView("clickstream_with_errors")

# Exclude rows that occur at or after the identified error timestamp
filtered_query = """
SELECT user_id,
       session_id,
       timestamp,
       event_type,
       event_page
FROM clickstream_with_errors
WHERE error_timestamp IS NULL OR timestamp < error_timestamp
ORDER BY user_id, session_id, timestamp
"""

filtered_df = spark.sql(filtered_query)
filtered_df.createOrReplaceTempView("filtered_clickstream")

# Use SQL to eliminate consecutive duplicates and create routes
route_query = """
WITH ranked_pages AS (
    SELECT user_id,
           session_id,
           event_page,
           ROW_NUMBER() OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS rn,
           LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS prev_page
    FROM filtered_clickstream
)
SELECT user_id,
       session_id,
       COLLECT_LIST(event_page) AS pages
FROM ranked_pages
WHERE event_page != prev_page OR prev_page IS NULL
GROUP BY user_id, session_id
"""

routes_df = spark.sql(route_query)
routes_df.createOrReplaceTempView("routes")

# Generate the final routes and count occurrences
count_routes_query = """
SELECT CONCAT_WS('-', pages) AS route,
       COUNT(*) AS count
FROM routes
GROUP BY route
ORDER BY count DESC
LIMIT 30
"""

result = spark.sql(count_routes_query)

# Show the final results
result.show()


2024-10-27 18:46:06,667 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
+--------------------+-----+
only showing top 20 rows



In [24]:
# Collect results to a list of tuples
result_list = result.collect()

# Format the results as a dictionary for JSON
results_dict = {route: count for route, count in result_list}

# Save the results to a JSON file
with open('result.json', 'w') as json_file:
    json.dump(results_dict, json_file)

In [26]:
#!curl -F file=@result.json 51.250.123.136:80/MDS-LSML1/nndrew/w6/2

In [27]:
spark.stop()

# Spark RDD solution

In [ ]:
# Setting up environment
import findspark
findspark.init()

# Spark imports
import pyspark
from pyspark import SparkContext

# Initialize SparkContext
sc = SparkContext(appName='jupyter')

# Load the CSV file into an RDD
clickstream_rdd = sc.textFile('/data/clickstream.csv')

# Parse the CSV data and create an RDD of tuples
header = clickstream_rdd.first()  # Get the header
clickstream_rdd = clickstream_rdd.filter(lambda line: line != header)  # Remove header

# Split the lines into fields
clickstream_rdd = clickstream_rdd.map(lambda line: line.split('\t'))

# Define a function to filter out error sessions and collect valid events
def process_session(events):
    valid_events = []
    error_found = False
    for event in events:
        user_id = int(event[0])
        session_id = int(event[1])
        event_type = event[2]
        event_page = event[3]
        timestamp = int(event[4])
        
        # Check for error event
        if 'error' in event_type:
            error_found = True
            break
        
        if not error_found:
            if valid_events and valid_events[-1][3] == event_page:  # Avoid duplicates
                continue
            valid_events.append((user_id, session_id, event_page, timestamp))
    
    return valid_events

# Group events by (user_id, session_id) and process each session
grouped_sessions = clickstream_rdd.map(lambda event: ((int(event[0]), int(event[1])), (event[3], int(event[4])))) \
    .groupByKey() \
    .mapValues(list)  # Convert to list for processing

# Process each session to filter errors and eliminate consecutive duplicates
valid_routes = grouped_sessions.flatMap(lambda x: process_session(x[1]))

# Create routes by session
session_routes = valid_routes.map(lambda x: ((x[0], x[1]), x[2]))  # (user_id, session_id) -> event_page
session_routes = session_routes.groupByKey()  # Group by user_id and session_id

# Build the final routes and count occurrences
final_routes = session_routes.map(lambda x: ('-'.join(x[1]), 1))  # Create route string
route_counts = final_routes.reduceByKey(lambda a, b: a + b)  # Count occurrences of each route

# Sort and get the top 30 routes
top_routes = route_counts.takeOrdered(30, key=lambda x: -x[1])  # Sort by count descending

# Show the final results
for route, count in top_routes:
    print(f"{route}\t{count}")


In [ ]:
spark.stop()